In [14]:
%env SPECTRAL_CONNECTIVITY_ENABLE_GPU=true

env: SPECTRAL_CONNECTIVITY_ENABLE_GPU=true


In [15]:
#import sys
#sys.path.append("../../lfp")

In [16]:
import os
os.chdir('/blue/npadillacoreano/mcum/SocialMemEphys/diff_fam_social_memory_ephys')



In [17]:

import cupy as xp
from cupyx.scipy.fft import ifft
from cupyx.scipy.sparse.linalg import svds
from spectral_connectivity import Multitaper, Connectivity
import importlib
import pandas as pd
from itertools import combinations
import os
from bidict import bidict
import lfp.lfp_analysis.LFP_collection as LFP_collection


In [21]:
df = pd.read_excel(r"lfp/channel_mapping_sme.xlsx")
spike_cols = [col for col in df.columns if 'spike_interface_' in col.lower()]

# Extract brain regions from column names
# Assumes format 'spike_interface_REGION'
brain_regions = [col.split('spike_interface_')[1] for col in spike_cols]

# Create nested dictionary
subject_to_channel_dict = {}

for _, row in df.iterrows():
    subject = row['Subject'].astype(str)
    # Initialize inner dictionary for this subject
    subject_to_channel_dict[subject] = {}
    
    # Populate inner dictionary with brain region: spike value pairs
    for col, region in zip(spike_cols, brain_regions):
        subject_to_channel_dict[subject][region] = int(row[col])
behavior_dicts = {}
def make_recording_to_subj_dict(data_path):
    recording_to_subject = {}
    for root, dirs, files in os.walk(data_path):
        for file in files:
            if file.endswith('merged.rec'):
                subject = str(int((file.split("_")[0]))/10)
                recording_to_subject[file] = subject
                behavior_dicts[file] = {}
    return recording_to_subject


def process(data_path):
    recording_to_subject = make_recording_to_subj_dict(data_path)
    print(recording_to_subject)
    collection = LFP_collection.LFPCollection(subject_to_channel_dict, data_path, recording_to_subject, 4)
    #collection.process()
    return collection
    
    
data_path = r"data/diff"

collection = process(data_path)


{'24_nov_p1_merged.rec': '2.4', '11_nov_p1_merged.rec': '1.1', '22_nov_p1_merged.rec': '2.2', '44_nov_p1_merged.rec': '4.4', '23_nov_p1_merged.rec': '2.3', '41_nov_p1_merged.rec': '4.1', '31_nov_p1_merged.rec': '3.1', '21_nov_p1_merged.rec': '2.1', '33_nov_p1_merged.rec': '3.3', '12_nov_p1_merged.rec': '1.2', '13_nov_p1_merged.rec': '1.3', '32_nov_p1_merged.rec': '3.2'}
Processing 24_nov_p1_merged.rec
Found first timestamp
Processing 11_nov_p1_merged.rec
Found first timestamp
Processing 22_nov_p1_merged.rec
Found first timestamp
Processing 44_nov_p1_merged.rec
Found first timestamp
Processing 23_nov_p1_merged.rec
Found first timestamp
Processing 41_nov_p1_merged.rec
Found first timestamp
Processing 31_nov_p1_merged.rec
Found first timestamp
Processing 21_nov_p1_merged.rec
Found first timestamp
Processing 33_nov_p1_merged.rec
Found first timestamp
Processing 12_nov_p1_merged.rec
Found first timestamp
Processing 13_nov_p1_merged.rec
Found first timestamp
Processing 32_nov_p1_merged.rec
F

In [22]:
collection.process()


  0%|          | 0/12 [00:00<?, ?it/s]

processing 24_nov_p1_merged.rec
RMS Traces calculated
Power Calculated
Coherence calcualatd


Maximum iterations reached. 3717 of 4681 converged
Maximum iterations reached. 3817 of 4681 converged
Maximum iterations reached. 3284 of 4681 converged
Maximum iterations reached. 4067 of 4681 converged
Maximum iterations reached. 3594 of 4681 converged
Maximum iterations reached. 2930 of 4681 converged
Maximum iterations reached. 3545 of 4681 converged
Maximum iterations reached. 3025 of 4681 converged
Maximum iterations reached. 3615 of 4681 converged
Maximum iterations reached. 3115 of 4681 converged
  8%|▊         | 1/12 [00:33<06:07, 33.43s/it]

Granger's causality calculated
processing 11_nov_p1_merged.rec
RMS Traces calculated
Power Calculated
Coherence calcualatd


Maximum iterations reached. 2862 of 4681 converged
Maximum iterations reached. 2493 of 4681 converged
Maximum iterations reached. 2912 of 4681 converged
Maximum iterations reached. 3138 of 4681 converged
Maximum iterations reached. 2357 of 4681 converged
Maximum iterations reached. 2953 of 4681 converged
Maximum iterations reached. 3049 of 4681 converged
Maximum iterations reached. 2449 of 4681 converged
Maximum iterations reached. 2617 of 4681 converged
Maximum iterations reached. 3230 of 4681 converged
 17%|█▋        | 2/12 [01:05<05:25, 32.60s/it]

...

Granger's causality calculated
processing 13_nov_p1_merged.rec
RMS Traces calculated
Power Calculated
Coherence calcualatd


Maximum iterations reached. 4199 of 4679 converged
Maximum iterations reached. 4398 of 4679 converged
Maximum iterations reached. 4072 of 4679 converged
Maximum iterations reached. 4245 of 4679 converged
Maximum iterations reached. 4199 of 4679 converged
Maximum iterations reached. 3885 of 4679 converged
Maximum iterations reached. 4043 of 4679 converged
Maximum iterations reached. 4082 of 4679 converged
Maximum iterations reached. 4250 of 4679 converged
Maximum iterations reached. 3953 of 4679 converged
 92%|█████████▏| 11/12 [05:53<00:32, 32.07s/it]

Granger's causality calculated
processing 32_nov_p1_merged.rec
RMS Traces calculated
Power Calculated
Coherence calcualatd


Maximum iterations reached. 3058 of 4680 converged
Maximum iterations reached. 2668 of 4680 converged
Maximum iterations reached. 3080 of 4680 converged
Maximum iterations reached. 3198 of 4680 converged
Maximum iterations reached. 2604 of 4680 converged
Maximum iterations reached. 2898 of 4680 converged
Maximum iterations reached. 2785 of 4680 converged
Maximum iterations reached. 2570 of 4680 converged
Maximum iterations reached. 2432 of 4680 converged
Maximum iterations reached. 2901 of 4680 converged
100%|██████████| 12/12 [06:25<00:00, 32.16s/it]

Granger's causality calculated


In [23]:
LFP_collection.LFPCollection.save_to_json(collection, r"data/diff_lfp")